# Moving away from the notebook

These Jupyter notebooks provide a nice code playground for looking at Python and testing individual ideas one by one but it is time to move away from this notebook and into an "*actual program*". You may have already been doing this with play.py file as we go along, but if not, it's time to open up a real program. 

What we're going to do is video, and we're going to do it with face detection to start off with. Open up (in a text editor or code editor...) the file 

```
start.py
```

This has the following code at the top of it (should look familiar...)

In [1]:
# these imports let you use opencv
import cv2 #opencv itself
import common #some useful opencv functions
import video # some video stuff
import numpy as np # matrix manipulations


For the purposes of the notebook (this page that you're reading now) we're going to have to do some notebook code in order to view images here

In [2]:
#the following are to do with this interactive notebook code
%matplotlib inline 
from matplotlib import pyplot as plt # this lets you draw inline pictures in the notebooks
import pylab # this allows you to control figure size 
pylab.rcParams['figure.figsize'] = (10.0, 8.0) # this controls figure size in the notebook

The other code that is in the file start.py opens the webcam (if the webcam works), then just puts the contents of the webcam in a window until someone presses escape, while counting up the frames (until someone presses escape).

Test this now. You should just have a really basic webcam viewer.

What we are going to do now is to edit this webcam viewer and introduce detections (from previous notebooks) and some video processing

## Coding exercise 1
Make your webcam image greyscale, and view that in the webcam viewer

## Coding exercise 2
Run a face detector on your grey webcam image, and draw the output as rectangles

## Coding exercise 3: motion detection

Exercise 3 is a little more challenging and involves motion detection.  We're going to do this the simplest way which is to keep a moving average of the colour in each frame, and then compare that to the current frame.

### 3a: setting up our constants

Methods often have parameters, and these can be set outside the main video loop. The parameters we're going to set are fbuffer which is a variable for the number of frames in our buffer. We'll start with 15, this means that if something is still for 15 frames it will disappear into the background.  alpha is the update rate for our background model and is a function of fbuffer. difference_thresh is the amount of difference from the background model that we're willing to accept before saying "*actually that object is moving*".

The final thing we set up is our actual moving average model, which is a variable called "movingaverage" the same size as the images we're getting, initialised with the first image (img).

```
fbuffer=15
alpha=float(1.0/fbuffer) 
difference_thresh=10
movingaverage=np.float32(img)
```


Copy these into the start.py file, outside of the main video loop.

### 3b: looking at the moving average model


Inside the video loop, we want to take each frame and update our moving average model. This is just one line of code in OpenCV (the first of the lines below) but we also want to take a look at it so we'll convert the scale so it'll show nicely as an image, and we'll copy it to output image. 

```
    cv2.accumulateWeighted(img,movingaverage,alpha) 
    res=cv2.convertScaleAbs(movingaverage)
    # show the background model 
    output_image=res.copy()  
```
Add these to the file (before the facedetect section - we don't want to incorporate the face rectangles into our background model!) and take a look at the background model. 

Experiment with different values of fbuffer

In [ ]:
    movingaverage=np.float32(img)
